<a href="https://www.kaggle.com/code/realshaktigupta/forecasting-competition?scriptVersionId=137878001" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import keras
import tensorflow.keras.backend as K
import datetime
from tqdm import tqdm
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor
from catboost import CatBoostRegressor
import matplotlib.pyplot as plt

In [2]:
#This notebooks is a work in progress and incomplete.

In [3]:
train=pd.read_csv("/kaggle/input/playground-series-s3e19/train.csv",index_col='id')
test=pd.read_csv("/kaggle/input/playground-series-s3e19/test.csv",index_col='id')

In [4]:
train.head()

,date,country,store,product,num_sold
id,,,,,
0,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Improve Your Coding,63
1,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Train More LLMs,66
2,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win Friends and Influence People,9
3,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win More Kaggle Competitions,59
4,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Write Better,49


In [5]:
len(train['date'].unique())

1826

In [6]:
cat_features=['country','store','product']
for feature in cat_features:
    print(feature,':',train[feature].unique())

country : ['Argentina' 'Canada' 'Estonia' 'Japan' 'Spain']
store : ['Kaggle Learn' 'Kaggle Store' 'Kagglazon']
product : ['Using LLMs to Improve Your Coding' 'Using LLMs to Train More LLMs'
 'Using LLMs to Win Friends and Influence People'
 'Using LLMs to Win More Kaggle Competitions' 'Using LLMs to Write Better']


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136950 entries, 0 to 136949
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   date      136950 non-null  object
 1   country   136950 non-null  object
 2   store     136950 non-null  object
 3   product   136950 non-null  object
 4   num_sold  136950 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 6.3+ MB


In [8]:
def datetime_features(df_temp):
    """
ref:-https://www.kaggle.com/code/chingiznurzhanov/timeseriessplit-catboost-trick
    """
    df_temp['month'] = df_temp['date'].dt.month
    df_temp['day'] = df_temp['date'].dt.day
    df_temp['year'] = df_temp['date'].dt.year
    df_temp['dayofweek'] = df_temp['date'].dt.dayofweek
    df_temp['quarter'] = df_temp['date'].dt.quarter
    df_temp['dayofmonth'] = df_temp['date'].dt.day
    #df_temp['weekofyear'] = df_temp['date'].dt.weekofyear
    df_temp['weekofyear'] = df_temp['date'].dt.isocalendar().week.astype(np.int64)
    df_temp['is_month_start'] = df_temp.date.dt.is_month_start.astype(np.int8)
    df_temp['is_month_end'] = df_temp.date.dt.is_month_end.astype(np.int8)
    df_temp['monday'] = df_temp.date.dt.weekday.eq(0).astype(np.uint8)
    df_temp['tuesday'] = df_temp.date.dt.weekday.eq(1).astype(np.uint8)
    df_temp['wednesday'] = df_temp.date.dt.weekday.eq(2).astype(np.uint8)
    df_temp['thursday'] = df_temp.date.dt.weekday.eq(3).astype(np.uint8)
    df_temp['friday'] = df_temp.date.dt.weekday.eq(4).astype(np.uint8)
    df_temp['saturday'] = df_temp.date.dt.weekday.eq(5).astype(np.uint8)
    df_temp['sunday'] = df_temp.date.dt.weekday.eq(6).astype(np.uint8)
    df_temp['is_quarter_end'] = df_temp['date'].dt.is_quarter_end.astype(np.uint8)
    df_temp['is_quarter_start'] = df_temp['date'].dt.is_quarter_start.astype(np.uint8)
    df_temp['is_year_end'] = df_temp['date'].dt.is_year_end.astype(np.uint8)
    df_temp['is_year_start'] = df_temp['date'].dt.is_year_start.astype(np.uint8)
    df_temp=df_temp.drop('date',axis=1)
    return df_temp

In [9]:
def data_encoding(dataset):
    cat_features=['country','store','product']
    for feature in tqdm(cat_features):
        dictionary={}
        n=1
        print('feature:',feature)
        print('categories:',dataset[feature].unique())
        for sub_cat in dataset[feature].unique():
            dictionary[sub_cat]=n
            n+=1
        print('encoding rule:',dictionary)
        dataset[feature]=dataset[feature].map(dictionary)
        dummies=pd.get_dummies(dataset[feature],prefix=feature)
        for key in dummies:
            dataset[key]=dummies[key]
        dataset=dataset.drop(feature,axis=1)
    dataset['date']=pd.to_datetime(dataset['date'],format='%Y-%m-%d')
    print("Date converted to Datetime features. Check returned dataframe if you want further details.")
    return datetime_features(dataset)

In [10]:
train=data_encoding(train)
test=data_encoding(test)

  0%|          | 0/3 [00:00<?, ?it/s]

feature: country
categories: ['Argentina' 'Canada' 'Estonia' 'Japan' 'Spain']
encoding rule: {'Argentina': 1, 'Canada': 2, 'Estonia': 3, 'Japan': 4, 'Spain': 5}
feature: store
categories: ['Kaggle Learn' 'Kaggle Store' 'Kagglazon']
encoding rule: {'Kaggle Learn': 1, 'Kaggle Store': 2, 'Kagglazon': 3}


 67%|██████▋   | 2/3 [00:00<00:00, 13.47it/s]

feature: product
categories: ['Using LLMs to Improve Your Coding' 'Using LLMs to Train More LLMs'
 'Using LLMs to Win Friends and Influence People'
 'Using LLMs to Win More Kaggle Competitions' 'Using LLMs to Write Better']
encoding rule: {'Using LLMs to Improve Your Coding': 1, 'Using LLMs to Train More LLMs': 2, 'Using LLMs to Win Friends and Influence People': 3, 'Using LLMs to Win More Kaggle Competitions': 4, 'Using LLMs to Write Better': 5}


100%|██████████| 3/3 [00:00<00:00, 12.35it/s]


Date converted to Datetime features. Check returned dataframe if you want further details.


100%|██████████| 3/3 [00:00<00:00, 47.60it/s]

feature: country
categories: ['Argentina' 'Canada' 'Estonia' 'Japan' 'Spain']
encoding rule: {'Argentina': 1, 'Canada': 2, 'Estonia': 3, 'Japan': 4, 'Spain': 5}
feature: store
categories: ['Kaggle Learn' 'Kaggle Store' 'Kagglazon']
encoding rule: {'Kaggle Learn': 1, 'Kaggle Store': 2, 'Kagglazon': 3}
feature: product
categories: ['Using LLMs to Improve Your Coding' 'Using LLMs to Train More LLMs'
 'Using LLMs to Win Friends and Influence People'
 'Using LLMs to Win More Kaggle Competitions' 'Using LLMs to Write Better']
encoding rule: {'Using LLMs to Improve Your Coding': 1, 'Using LLMs to Train More LLMs': 2, 'Using LLMs to Win Friends and Influence People': 3, 'Using LLMs to Win More Kaggle Competitions': 4, 'Using LLMs to Write Better': 5}
Date converted to Datetime features. Check returned dataframe if you want further details.


In [11]:
train.head()

,num_sold,country_1,country_2,country_3,country_4,country_5,store_1,store_2,store_3,product_1,...,tuesday,wednesday,thursday,friday,saturday,sunday,is_quarter_end,is_quarter_start,is_year_end,is_year_start
id,,,,,,,,,,,,,,,,,,,,,
0,63,1,0,0,0,0,1,0,0,1,...,0,0,0,0,0,1,0,1,0,1
1,66,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,1,0,1
2,9,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,1,0,1
3,59,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,1,0,1
4,49,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,1,0,1


In [12]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136950 entries, 0 to 136949
Data columns (total 34 columns):
 #   Column            Non-Null Count   Dtype
---  ------            --------------   -----
 0   num_sold          136950 non-null  int64
 1   country_1         136950 non-null  uint8
 2   country_2         136950 non-null  uint8
 3   country_3         136950 non-null  uint8
 4   country_4         136950 non-null  uint8
 5   country_5         136950 non-null  uint8
 6   store_1           136950 non-null  uint8
 7   store_2           136950 non-null  uint8
 8   store_3           136950 non-null  uint8
 9   product_1         136950 non-null  uint8
 10  product_2         136950 non-null  uint8
 11  product_3         136950 non-null  uint8
 12  product_4         136950 non-null  uint8
 13  product_5         136950 non-null  uint8
 14  month             136950 non-null  int64
 15  day               136950 non-null  int64
 16  year              136950 non-null  int64
 17  dayofweek 

In [13]:
X,y=train.drop("num_sold",axis=1),train["num_sold"]

In [14]:
#(tf.convert_to_tensor(X),tf.convert_to_tensor(y))

In [15]:
# cb_reg=CatBoostRegressor(loss_function='MAPE',verbose=3)
# cb_reg.fit(X,y)

In [16]:
# bag_reg=BaggingRegressor(cb_reg,n_estimators=5,n_jobs=-1,verbose=3)
# bag_reg.fit(X,y)

In [17]:
#bag_reg.predict(test)

In [18]:
#cb_reg.predict(test)

In [19]:
# rf_reg=RandomForestRegressor(n_estimators=1000,n_jobs=-1,verbose=3)
# rf_reg.fit(X,y)

In [20]:
def min_max_scaling(df):
    df_scaled = df.copy()
    for column in df_scaled.columns:
        df_scaled[column]=(df_scaled[column]-df_scaled[column].min())/(df_scaled[column].max()-df_scaled[column].min()+1e-16)    
    return(df_scaled)

In [21]:
X=min_max_scaling(X)
test=min_max_scaling(test)

In [22]:
X_train=X[:round(0.9*len(X))]
X_val=X[round(0.9*len(X)):]
y_train=y[:round(0.9*len(X))]
y_val=y[round(0.9*len(X)):]

In [23]:
X_train_tf=tf.convert_to_tensor(X_train)
X_val_tf=tf.convert_to_tensor(X_val)
y_train_tf=tf.convert_to_tensor(y_train.astype(np.float32))
y_val_tf=tf.convert_to_tensor(y_val.astype(np.float32))
test_tf=tf.convert_to_tensor(test)

In [24]:
test_tf

<tf.Tensor: shape=(27375, 33), dtype=float64, numpy=
array([[1., 0., 0., ..., 1., 0., 1.],
       [1., 0., 0., ..., 1., 0., 1.],
       [1., 0., 0., ..., 1., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])>

In [25]:
train_tf=tf.data.Dataset.from_tensor_slices((X_train_tf,y_train_tf)).prefetch(600)
val_tf=tf.data.Dataset.from_tensor_slices((X_val_tf,y_val_tf)).prefetch(600)

In [26]:
len(test_tf)

27375

In [27]:
#zeroes=tf.zeros(shape=(27375,),dtype=tf.dtypes.float32)

In [28]:
test_tf=tf.data.Dataset.from_tensor_slices(test_tf)

In [29]:
test_tf

<_TensorSliceDataset element_spec=TensorSpec(shape=(33,), dtype=tf.float64, name=None)>

In [30]:
strategy=tf.distribute.MirroredStrategy()

In [31]:
with strategy.scope():
    def smape_loss(y_true, y_pred):
        epsilon = 0.1
        summ = K.maximum(K.abs(y_true) + K.abs(y_pred) + epsilon, 0.5 + epsilon)
        smape = K.abs(y_pred - y_true) / summ * 2.0
        return smape

In [119]:
with strategy.scope():
    model=tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=(33,)))
    model.add(tf.keras.layers.Dense(768,activation='relu',activity_regularizer=tf.keras.regularizers.l1_l2(0.1,0.15)))
    model.add(tf.keras.layers.BatchNormalization())
#     model.add(tf.keras.layers.Dense(2048,activation='relu',activity_regularizer=tf.keras.regularizers.l1_l2(0.01,0.15)))
#     model.add(tf.keras.layers.BatchNormalization())
#     model.add(tf.keras.layers.Dense(1024,activation='relu',activity_regularizer=tf.keras.regularizers.l1(0.9)))
#     model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(512,activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.4))
    model.add(tf.keras.layers.Dense(256,activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(64,activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.4))
    model.add(tf.keras.layers.Dense(10,activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(1,activation='linear',activity_regularizer=tf.keras.regularizers.l1(0.1)))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),loss="mean_absolute_percentage_error",metrics=[smape_loss])

In [120]:
model.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_97 (Dense)            (None, 768)               26112     
                                                                 
 batch_normalization_74 (Bat  (None, 768)              3072      
 chNormalization)                                                
                                                                 
 dense_98 (Dense)            (None, 512)               393728    
                                                                 
 batch_normalization_75 (Bat  (None, 512)              2048      
 chNormalization)                                                
                                                                 
 dropout_32 (Dropout)        (None, 512)               0         
                                                                 
 dense_99 (Dense)            (None, 256)             

In [121]:
with strategy.scope():
    early_stopping=tf.keras.callbacks.EarlyStopping(monitor="val_loss",min_delta=0.01,patience=10,verbose=1,mode="min",
                                                    restore_best_weights=True,start_from_epoch=10)
    checkpoint=tf.keras.callbacks.ModelCheckpoint("/kaggle/working/mc_forecast_regressor.h5",monitor="val_loss",
                                                  save_best_only=True,verbose=1,mode='min',save_freq='epoch')
    lr=tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",factor=0.5,patience=3,verbose=3,mode="min",min_delta=0.01,
                                            cooldown=0,min_lr=0)

In [122]:
with strategy.scope():
    history=model.fit(train_tf.batch(128),epochs=300,callbacks=[early_stopping,checkpoint,lr],
              validation_data= val_tf.batch(128),use_multiprocessing=True,shuffle=False)

Epoch 1/300
959/963 [============================>.] - ETA: 0s - loss: 44.6733 - smape_loss: 0.5979
Epoch 1: val_loss improved from inf to 37.31832, saving model to /kaggle/working/mc_forecast_regressor.h5
963/963 [==============================] - 16s 11ms/step - loss: 44.6304 - smape_loss: 0.5970 - val_loss: 37.3183 - val_smape_loss: 0.3747 - lr: 0.0100
Epoch 2/300
958/963 [============================>.] - ETA: 0s - loss: 32.2582 - smape_loss: 0.2879
Epoch 2: val_loss improved from 37.31832 to 34.23555, saving model to /kaggle/working/mc_forecast_regressor.h5
963/963 [==============================] - 10s 10ms/step - loss: 32.2509 - smape_loss: 0.2877 - val_loss: 34.2355 - val_smape_loss: 0.2785 - lr: 0.0100
Epoch 3/300
960/963 [============================>.] - ETA: 0s - loss: 30.7228 - smape_loss: 0.2376
Epoch 3: val_loss improved from 34.23555 to 33.27616, saving model to /kaggle/working/mc_forecast_regressor.h5
963/963 [==============================] - 11s 11ms/step - loss: 30.

In [123]:
with strategy.scope():
    model=keras.models.load_model("/kaggle/working/mc_forecast_regressor.h5",compile=False)
    predictions=model.predict(test_tf.batch(len(test_tf)))

1/1 [==============================] - 0s 348ms/step


In [124]:
predictions

array([[1382.7616  ],
       [1260.9078  ],
       [  14.116077],
       ...,
       [  88.62541 ],
       [ 554.91724 ],
       [ 495.57608 ]], dtype=float32)

In [125]:
list(predictions.reshape(1,-1)[0])

[1382.7616,
 1260.9078,
 14.116077,
 502.70288,
 31.6872,
 2513.8577,
 2574.547,
 1158.1532,
 2046.8783,
 1425.8939,
 188.09924,
 177.6687,
 34.89751,
 183.78519,
 174.40808,
 220.59396,
 211.46788,
 29.38467,
 102.914345,
 99.9292,
 248.35332,
 238.632,
 30.574877,
 167.7628,
 147.08783,
 405.36285,
 392.09003,
 243.61224,
 234.89235,
 356.92273,
 91.68493,
 88.51285,
 16.473684,
 79.765724,
 66.84691,
 108.876,
 106.09629,
 25.00921,
 99.41576,
 95.434296,
 283.37796,
 461.52258,
 81.59016,
 489.26984,
 397.7784,
 102.822,
 100.989746,
 22.996851,
 96.68368,
 88.126,
 164.78914,
 162.93205,
 29.248367,
 149.78075,
 112.55957,
 371.74548,
 360.03256,
 104.835495,
 458.84122,
 587.18274,
 96.39489,
 93.4406,
 17.560251,
 84.51225,
 70.14068,
 110.8651,
 107.52905,
 27.115286,
 100.12909,
 100.65731,
 579.9767,
 582.512,
 87.75975,
 540.89966,
 499.9911,
 36.547436,
 37.82216,
 6.5028872,
 31.752356,
 32.02856,
 54.86622,
 58.254898,
 9.024609,
 52.659065,
 46.298866,
 153.84833,
 183.8

In [128]:
submission=pd.DataFrame(index=test.index)
#submission['id']=test.index
num_sold=[round(x) for x in list(predictions.reshape(1,-1)[0])]
submission['num_sold']=num_sold
submission.to_csv("submission.csv",index='id')

In [129]:
submission

,num_sold
id,
136950,1383
136951,1261
136952,14
136953,503
136954,32
...,...
164320,598
164321,601
164322,89


In [130]:
pd.read_csv("/kaggle/working/submission.csv")

,id,num_sold
0,136950,1383
1,136951,1261
2,136952,14
3,136953,503
4,136954,32
...,...,...
27370,164320,598
27371,164321,601
27372,164322,89
27373,164323,555


In [ ]:
#ensemble or blenders can be a good idea?